In [1]:
# Dependencies
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.request
import requests
import pandas as pd
import time
import re
import json
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
from sqlalchemy import create_engine

In [2]:
csv_hospital = "Healthcare_Associated_Infections-Hospital.csv"
hospital_df = pd.read_csv(csv_hospital)
hospital_df.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Score,Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,NaN,07/01/2019,12/31/2020
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.230,NaN,07/01/2019,12/31/2020
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,NaN,07/01/2019,12/31/2020
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,NaN,07/01/2019,12/31/2020
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,NaN,07/01/2019,12/31/2020


In [3]:
hospital_df.count()

Facility ID             174780
Facility Name           174780
Address                 174780
City                    174780
State                   174780
ZIP Code                174780
County Name             174780
Phone Number            174780
Measure ID              174780
Measure Name            174780
Compared to National    174780
Score                   174780
Footnote                 82415
Start Date              174780
End Date                174780
dtype: int64

In [4]:
#Footnotes column not needed. Drop column name 'Footnote' from Data Frame

hospital_df2 = hospital_df.drop(['Footnote'], axis = 1)

In [5]:
hospital_df2.count()

Facility ID             174780
Facility Name           174780
Address                 174780
City                    174780
State                   174780
ZIP Code                174780
County Name             174780
Phone Number            174780
Measure ID              174780
Measure Name            174780
Compared to National    174780
Score                   174780
Start Date              174780
End Date                174780
dtype: int64

In [6]:
#Count of records per state
hospital_df2.State.value_counts()

TX    14436
CA    12240
FL     6732
IL     6480
NY     6120
PA     6084
OH     6048
GA     4932
MI     4860
KS     4860
MN     4644
WI     4644
OK     4356
IN     4320
LA     4320
IA     4212
MO     4032
NC     3960
TN     3708
MS     3456
WA     3384
KY     3384
AL     3312
NE     3276
VA     3168
CO     3132
AZ     2988
AR     2844
NJ     2412
MA     2340
MT     2268
SC     2232
OR     2232
SD     2160
PR     1944
WV     1836
MD     1836
UT     1728
ND     1620
NM     1584
ID     1584
NV     1368
ME     1224
CT     1116
WY     1008
NH      936
HI      864
AK      828
VT      540
RI      396
DC      288
DE      288
VI       72
GU       72
AS       36
MP       36
Name: State, dtype: int64

In [7]:
# Rename Columns

hospital_df3 = hospital_df2.rename(columns={'Facility ID':'facility_id',
                                 'Facility Name':'name',
                                 'Address':'address',
                                 'City':'city',
                                 'State':'state',
                                 'ZIP Code':'zip',
                                 'County Name':'county',
                                 'Phone Number':'phone',
                                 'Measure ID':'measure_id',
                                 'Measure Name':'measure_name',
                                 'Compared to National':'comparison',
                                 'Score':'score',
                                 'Start Date':'start',
                                 'End Date':'end'}
          )

In [8]:
hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,07/01/2019,12/31/2020
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.230,07/01/2019,12/31/2020
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,07/01/2019,12/31/2020
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,07/01/2019,12/31/2020
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,07/01/2019,12/31/2020


In [9]:
# Add an id column (the index)

hospital_df3["id"] = hospital_df3.index + 1

hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end,id
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,07/01/2019,12/31/2020,1
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.230,07/01/2019,12/31/2020,2
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,07/01/2019,12/31/2020,3
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,07/01/2019,12/31/2020,4
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,07/01/2019,12/31/2020,5


In [11]:
# Set index
hospital_df3.set_index()

hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end
id,,,,,,,,,,,,,,
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,07/01/2019,12/31/2020
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.230,07/01/2019,12/31/2020
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,07/01/2019,12/31/2020
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,07/01/2019,12/31/2020
5,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,07/01/2019,12/31/2020
